In [1]:
from torch import nn
import torch.nn.functional as F
import torch
INPUT_DIM = 101 # X is 101-dimensional
HIDDEN_DIM = INPUT_DIM * 4 # Center-most latent space vector will have length of 404
NUM_CLASSES = 15 # 16 classes

print(INPUT_DIM)
print(HIDDEN_DIM)
print(NUM_CLASSES)


101
404
15


In [2]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim * 2)
        self.fc2 = nn.Linear(input_dim * 2, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, int(hidden_dim / 2))
        self.fc4 = nn.Linear(int(hidden_dim / 2), int(hidden_dim / 4))
        self.fc5 = nn.Linear(int(hidden_dim / 4), int(hidden_dim / 8))
        self.fc6 = nn.Linear(int(hidden_dim / 8), num_classes)

    def forward(self, x_in):
        z = F.relu(self.fc1(x_in))  # ReLU activation function added!
        z = F.relu(self.fc2(z))
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        z = self.fc6(z)
        return z


# Initialize model
model = MLP(input_dim=INPUT_DIM, hidden_dim=HIDDEN_DIM, num_classes=NUM_CLASSES)
print(model.named_parameters)

<bound method Module.named_parameters of MLP(
  (fc1): Linear(in_features=101, out_features=202, bias=True)
  (fc2): Linear(in_features=202, out_features=404, bias=True)
  (fc3): Linear(in_features=404, out_features=202, bias=True)
  (fc4): Linear(in_features=202, out_features=101, bias=True)
  (fc5): Linear(in_features=101, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=15, bias=True)
)>


In [3]:
checkpoint = torch.load('model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

MLP(
  (fc1): Linear(in_features=101, out_features=202, bias=True)
  (fc2): Linear(in_features=202, out_features=404, bias=True)
  (fc3): Linear(in_features=404, out_features=202, bias=True)
  (fc4): Linear(in_features=202, out_features=101, bias=True)
  (fc5): Linear(in_features=101, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=15, bias=True)
)

In [18]:
# Inputs for inference

import os
import pandas as pd

current_folder = globals()['_dh'][0]
csv_path = os.path.join(current_folder, 'training_set.csv')
df = pd.read_csv(csv_path, header=0) # load
hi = '0.6276978850364685,0.5843755602836609,-0.22295062243938446,0.6268095374107361,0.5737197995185852,-0.20632241666316986,0.626587986946106,0.5743216872215271,-0.20632290840148926,0.6263055801391602,0.5748404860496521,-0.20634683966636658,0.624580979347229,0.5716469883918762,-0.23019017279148102,0.6226451396942139,0.5708016157150269,-0.2302272915840149,0.6204419732093811,0.569793701171875,-0.23026911914348602,0.6164519190788269,0.5789487361907959,-0.11819145828485489,0.60848069190979,0.5737546682357788,-0.2259415239095688,0.6218960881233215,0.6019092202186584,-0.18988178670406342,0.6198330521583557,0.5968146324157715,-0.22149603068828583,0.569865882396698,0.6342253088951111,0.013678950257599354,0.5751160979270935,0.654432475566864,-0.28310051560401917,0.5595629811286926,0.7733974456787109,0.015326440334320068,0.5656692981719971,0.7923648357391357,-0.32690662145614624,0.5588405132293701,0.8882727026939392,-0.054843779653310776,0.5610483288764954,0.930568277835846,-0.3327784538269043,0.5656618475914001,0.8991759419441223,-0.04507143795490265,0.5689927935600281,0.9615245461463928,-0.3773484528064728,0.5691770911216736,0.9094014763832092,-0.08489023894071579,0.5727695226669312,0.9585964679718018,-0.35044315457344055,0.5621487498283386,0.9082633256912231,-0.0723174437880516,0.567960798740387,0.9476576447486877,-0.3250153958797455,0.47402092814445496,0.7038294672966003,0.08810194581747055,0.45962613821029663,0.7189508676528931,-0.08809611946344376,0.5689114928245544,0.7234283685684204,0.16465863585472107,0.3733466565608978,0.8305726051330566,-0.11438359320163727,0.5591745376586914,0.8977935910224915,0.24112708866596222,0.2668099105358124,0.8921425938606262,-0.06320709735155106,0.5496239066123962,0.9240500926971436,0.24600288271903992,0.253009170293808,0.8905003666877747,-0.059814102947711945,0.5976037383079529,0.9380369782447815,0.23735664784908295,0.2811347544193268,0.9561828374862671,-0.11795039474964142,0,1,0'
ok = hi.split(',')

column_headers = df.columns.tolist()
column_headers.pop()

sample = [{header: ok[i]} for i, header in enumerate(column_headers)]

X_infer = pd.DataFrame(sample) # all 101(or 102?) features

# Standardize
from sklearn.preprocessing import StandardScaler
# Standardize the data (mean=0, std=1) using training data
X_scaler = StandardScaler().fit(X_train)

X_infer = X_scaler.transform(X_infer)
print(X_infer)
# Predict
y_infer = F.softmax(model(torch.Tensor(X_infer)), dim=1)
prob, _class = y_infer.max(dim=1)
label = label_encoder.inverse_transform(_class.detach().numpy())[0]
print (f"The probability that you have {label} is {prob.detach().numpy()[0]*100}.")


NameError: name 'X_scaler' is not defined